In [101]:
import pandas as pd
import numpy as np

In [102]:
train = pd.read_csv('../raw_data/train_1M.csv')
questions = pd.read_csv('../data/questions.csv')

In [103]:
# Only keep questions, rename content_id and merge with questions table.
train = train.query('content_type_id==0')
train = train.rename(columns={'content_id': 'question_id'})
train = train.merge(questions, on='question_id', how='left')
# Add features.
train['user_activity_cumcount']=train.groupby('user_id').cumcount()
train['question_avg_score'] = train.groupby('question_id')['answered_correctly'].mean()
train['avgscore_cummean'] = train.groupby('user_id').answered_correctly.apply(lambda x: x.shift().expanding().mean())
# Then remove all columns but 'question_avg_score', 'user_avgscore_cummean', 'answered_correctly' (target). Replace inf by 0
b0_train = train[['question_avg_score', 'avgscore_cummean', 'answered_correctly']]
b0_train.avgscore_cummean.replace(np.nan, 0, inplace=True)

In [106]:
b0_train.to_csv('b0_model.csv')